In [1]:
#!/usr/bin/env python
import rospy
from human_moveit_config.human_model import HumanModel
import json
import rospkg
import time
from sensor_msgs.msg import JointState
from os.path import join
from os.path import exists
from baxter_commander.persistence import dicttostate
import sys
from reba_optim.reba_assess import RebaAssess
from os import listdir
from os.path import isdir
from numpy.linalg import norm
from numpy import array
from keras.models import Sequential, model_from_json
from keras.layers.core import Dense, Activation, Dropout
import numpy as np
import time

Using Theano backend.


In [2]:
rospy.init_node('assess')
rospack = rospkg.RosPack()

In [3]:
res_folder = 'results'

In [4]:
reba = RebaAssess()

In [5]:
# @Baptiste, if you wanna load the model I trained 
# and play around with it, use those two lines, 
# instead of the block way up above, 
# with the model=Sequential()... and so on

modelName = "bb-abc"
modelVersion = 1

model = model_from_json(open("model-"+modelName+'-'+str(modelVersion)+'.json').read())
model.load_weights("model-"+modelName+'-'+str(modelVersion)+'.h5')

In [6]:
joint_names = ['neck_0', 'neck_1', 'neck_2', 'spine_0', 'spine_1', 'spine_2', 'right_knee',
               'left_knee', 'right_shoulder_0', 'right_shoulder_1', 'left_shoulder_0',
               'left_shoulder_1', 'right_elbow_0', 'left_elbow_0', 'right_wrist_0',
               'right_wrist_1', 'left_wrist_0', 'left_wrist_1']
reba_joints = ['neck_0', 'neck_1', 'neck_2', 'spine_0', 'spine_1', 'spine_2',
               'right_shoulder_0', 'right_shoulder_1', 'right_shoulder_2',
               'right_elbow_0', 'right_elbow_1', 'right_wrist_0', 'right_wrist_1']

In [7]:
def distance(poseA, poseB):
    posA = array(poseA[0])
    posB = array(poseB[0])
    return norm(posA - posB)

In [8]:
def dist_with_expected(recorded, expected, data):
    for j in reba_joints:
        rec = recorded.position[recorded.name.index(j)]
        exp = expected[recorded.name.index(j)]
        data['distance_to_expected']['reba'][j].append(abs(rec - exp))

In [9]:
def assess(id_user, condition, shape, epsilon=0.1):
    # import file
    dir_results = join(rospack.get_path("thr_xp_human_comfort"), res_folder)
    filename = join(dir_results, id_user, 'records', condition, shape, 'replay.json')

    if exists(filename):
        with open(filename) as data_file:
            data = json.load(data_file)
    
        data['posture_score'] = {}
        data['posture_score']['data'] = []
        data['posture_score']['data_per_joint'] = {}
        for j in joint_names:
            data['posture_score']['data_per_joint'][j] = []
        data['posture_score']['insertion_index'] = []
        
        # detect insertion time
        start = -1
        end = -1
        for i in range(len(data['object_poses'])):
            dict_state = data['states'][i]
            # get the state
            state = dicttostate(dict_state)
            
            #score = reba.assess_posture(state.joint_state.position, state.joint_state.name)
            X = []
            for j in joint_names:
                X.append(state.joint_state.position[state.joint_state.name.index(j)])
            score = model.predict([np.array([X])])
            data['posture_score']['data'].append(score[0][0])
            # get the data for all joints
            reba.assess_posture_close(state.joint_state.position, state.joint_state.name,
                                      data['posture_score']['data_per_joint'])
            
            shapeo_pose = data['object_poses'][i]['shapeo']
            try:
                hand_pose = data['records'][i]['right_hand']
                if distance(shapeo_pose, hand_pose) < epsilon and start == -1:
                    start = i
                if distance(shapeo_pose, hand_pose) > epsilon and start != -1:
                    end = i
            except:
                pass

        if end == -1 and data['object_poses']:
            end = len(data['object_poses'])
            
        data['posture_score']['insertion_index'] = [start, end]
        
        # check the distance between expected and recorded pose
        data['distance_to_expected'] = {}
        data['distance_to_expected']['reba'] = {}
        for j in reba_joints:
            data['distance_to_expected']['reba'][j] = []
        if condition == 'reba':
            with open(join(dir_results, id_user, 'poses', 'reba_pose.json')) as datafile:
                expected_pose = json.load(datafile)['state']['position']
                
            for i in range(start, end):
                dict_state = data['states'][i]
                # get the state
                state = dicttostate(dict_state).joint_state
                
                # calculate the distance
                dist_with_expected(state, expected_pose, data)

        # dump the json file
        #with open('/tmp/test.json', 'w') as outfile:
        with open(filename, 'w') as outfile:
            json.dump(data, outfile, indent=4, sort_keys=True)
    else:
        print 'file ' + filename + ' not calculated yet'

In [10]:
#assess('6127', 'reba', 'hexagon', 0.1)

In [12]:
directory = join(rospack.get_path("thr_xp_human_comfort"), res_folder)
user_list = [name for name in listdir(directory) if isdir(join(directory, name))]

# loop through all the users
for user in user_list:

    # get dictionnary for results
    dir_results = join(directory, user, 'records')
    conditions = [name for name in listdir(dir_results) if isdir(join(dir_results, name))]
    shapes = ['star', 'hexagon', 'ellipse', 'slice', 'trapezium']

    for c in conditions:
        for s in shapes:
            if rospy.is_shutdown():
                sys.exit(0)
            assess(user, c, s, 0.2)

file /home/buschbapti/catkin_ws/src/thr_infrastructure/thr_experiments/thr_xp_human_comfort/results/449/records/reba/star/replay.json not calculated yet
file /home/buschbapti/catkin_ws/src/thr_infrastructure/thr_experiments/thr_xp_human_comfort/results/449/records/reba/hexagon/replay.json not calculated yet
file /home/buschbapti/catkin_ws/src/thr_infrastructure/thr_experiments/thr_xp_human_comfort/results/449/records/reba/ellipse/replay.json not calculated yet
file /home/buschbapti/catkin_ws/src/thr_infrastructure/thr_experiments/thr_xp_human_comfort/results/449/records/reba/slice/replay.json not calculated yet
file /home/buschbapti/catkin_ws/src/thr_infrastructure/thr_experiments/thr_xp_human_comfort/results/449/records/reba/trapezium/replay.json not calculated yet
file /home/buschbapti/catkin_ws/src/thr_infrastructure/thr_experiments/thr_xp_human_comfort/results/449/records/relative/star/replay.json not calculated yet
file /home/buschbapti/catkin_ws/src/thr_infrastructure/thr_experim